In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from torch.utils.data import DataLoader, TensorDataset

def make_dataloader(n):
    X = np.random.random((n, 2)) * 2 - 1
    Y = (X[:, 0] > X[:, 1] + 0.25).astype(int) + 1

    X = torch.tensor(X, dtype=torch.float)
    Y = torch.tensor(Y, dtype=torch.long)

    div1 = int(n*0.8)
    div2 = int(n*0.9)
    Xs = [X[:div1], X[div1:div2], X[div2:]]
    Ys = [Y[:div1], Y[div1:div2], Y[div2:]]

    dataset = TensorDataset(Xs[0], Ys[0])
    data_loader = DataLoader(dataset, batch_size=4)
    return data_loader

In [3]:
import torch.nn as nn
from metal.mmtl.task import Task

foo_input = nn.Linear(2, 10)
bar_input = foo_input #nn.Linear(100, 7)

foo_head = nn.Linear(10, 2)
bar_head = nn.Linear(10, 2)

foo = Task("foo_task", [make_dataloader(5000), None, None], foo_input, foo_head)
bar = Task("bar_task", [make_dataloader(2000), None, None], bar_input, bar_head)
# baz = Task("baz_task", "baz_head", [make_dataloader(100), None, None])
tasks = [foo, bar]

In [4]:
from metal.end_model import EndModel
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.trainer import MultitaskTrainer

model = MetalModel(tasks)
# trainer = MultitaskTrainer()
# trainer.train_model(model, tasks, n_epochs=1)


Network architecture:
MetalModel(
  (input_modules): ModuleDict(
    (bar_task): Linear(in_features=2, out_features=10, bias=True)
    (foo_task): Linear(in_features=2, out_features=10, bias=True)
  )
  (head_modules): ModuleDict(
    (bar_task): Linear(in_features=10, out_features=2, bias=True)
    (foo_task): Linear(in_features=10, out_features=2, bias=True)
  )
  (task_paths): ModuleDict(
    (bar_task): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
    (foo_task): Sequential(
      (0): Linear(in_features=2, out_features=10, bias=True)
      (1): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)



In [10]:
for batch in foo.data_loaders[0]:
    X, Y = batch
    print(model(X, ['foo_task']))
    print(model.calculate_loss(X, Y, ['foo_task']))    
    print(model.calculate_output(X, ['foo_task']))    
    break

{'foo_task': tensor([[ 0.5948, -0.1845],
        [ 0.4917,  0.0879],
        [ 0.6504, -0.1776],
        [ 0.5211, -0.0377]], grad_fn=<AddmmBackward>)}
{'foo_task': tensor(0.4260)}
{'foo_task': tensor([[0.6855, 0.3145],
        [0.5996, 0.4004],
        [0.6959, 0.3041],
        [0.6362, 0.3638]])}
